In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('data/liss_train.csv')

In [ ]:
df.columns

In [ ]:
df.head()

## Feature selection.

This needs manual inspection of the codebook and manually selection of features. Some issues:
- Categorical features are denoted with numbers
- Features on a scale have value 999 for other / unknown
- Some features are combinations of other features, but not quite (e.g. status, contract and status.contract)
- There are missing values all over the place

In [ ]:
columns_21 = df.columns[df.columns.str.endswith('21')]
columns_21

In [ ]:
columns_11 =  df.columns[df.columns.str.endswith('11')]
columns_11

In [ ]:
features_11 = df[feature_names]

In [ ]:
features_11.describe()

In [ ]:
features.dtypes

In [ ]:
features.nunique()

In [ ]:
numerical_features = ['age.11', 
                      'bruttoinc_imputed.11',
                      'financial.satisfaction.11',
                      'happiness.11',
                      'life.satisfaction.11',
                      'social.trust.11',
                      'gov.satisfaction.11',
                     ]

categorical_features = ['contract.11',
                        'educ.no.dipl.11',
                        'educ.dipl.11',
                        'domestic.status.11',
                        'civil.status.11',
                        'interest.news.11',
                        'last.vote.11'
                       ]

In [ ]:
df[numerical_features].describe()

Replace 999s with NaN:

In [ ]:
import numpy as np

X_num = df[numerical_features]
for feat in numerical_features[2:]:
    X_num[feat] = X_num[feat].replace(999, np.NaN)

In [ ]:
X_num.describe()

## Classification
Issues:
- Some values status.contract.21 are quite rare
- There are missing values in status.contract.21 even though status is known

In [ ]:
y = df['status.contract.21']

In [ ]:
# import seaborn as sns

data_plot = X_num.copy()
data_plot['status.contract.21'] = df['status.contract.21']
sns.pairplot(data_plot, hue='status.contract.21')

In [ ]:
y.describe()

In [ ]:
y.value_counts(dropna=False)

In [ ]:
# Replace unknowns in target
y = y.fillna('unknown')

In [ ]:
# Some classes are quite rare. Combine these
df['status.21'].value_counts(dropna=False)

In [ ]:
df.fillna('unknown').groupby(['status.21', 'status.contract.21']).count()['age.21']

For now we only take the status as target

In [ ]:
y = df['status.21']

In [ ]:
# Split train test
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_num, y, test_size=0.1, random_state=0)

Let's use the HistGradientBoosting classifier, it automatically handles missing values.

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier

In [ ]:
cls = HistGradientBoostingClassifier()

In [ ]:
cls.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

ConfusionMatrixDisplay.from_estimator(cls, X_test, y_test)

In [ ]:
import sklearn.metrics

y_test_pred = cls.predict(X_test)

print(sklearn.metrics.classification_report(y_test, y_test_pred))

## Regression
Choose income as target. Issues here:
- Target is sometimes missing

In [ ]:
# the target is sometimes missing, remove those rows
df_sub = df[~pd.isna(df['bruttoinc_imputed.21'])]

y_r = df_sub['bruttoinc_imputed.21']

# Select the numerical features
X_num = df_sub[numerical_features]
for feat in numerical_features[2:]:
    X_num[feat] = X_num[feat].replace(999, np.NaN)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_num, y_r, test_size=0.1, random_state=0)

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor

In [ ]:
est = HistGradientBoostingRegressor().fit(X_train, y_train)

In [ ]:
print('train: ', est.score(X_train, y_train))
print('test: ', est.score(X_test, y_test))

In [ ]:
y_test_pred = est.predict(X_test)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(y_test, y_test_pred)